In [17]:
# %load_ext autoreload
# %autoreload 2

# 创建文档
from docx import Document

# 其他
from pyautoword import *
import pandas as pd
# to_markdown需要安装一个包tabulate

In [2]:
# 变量
xxmc = '西安工业大学'
year = '2022'
# 额外分组，比如分学历，分师范生：
other_groupby = 'xl'

# 取数

In [3]:
# 获取派遣数据
conn = get_yjy_db('dw_database')
sql =f'''
    SELECT * FROM dw_s_employment_auto
    where xxmc = '{xxmc}' and substring(bynd,1,4) = '{year}'
    '''
data = pd.read_sql(sql=sql, con=conn)
data.head(1)

,xxdm,xxmc,bynd,yx,zydm,zy,bj,xm,xh,sfzhm,...,dim_company_property,dim_industry_category,dim_scale,dim_registered_capital,dim_dwmc_province,dim_dwmc_city,jylbfl,dwhy_c,sjly,xxmc_c
0,10702,西安工业大学,2022,计算机科学与工程学院,082107,信息对抗技术,None,卓恒,17060315120,610431199810142213,...,,,,,,,签就业协议形式就业,信息传输、软件和信息技术服务业,国库,None


# 获取分组表格

In [24]:
%aimport pyautoword
string = 'yx'

In [25]:
data1 = groupby_data(data,string,other_groupby)

,yx,num0,proportion0,num1,proportion1,num2,proportion2
0,中国书法学院,62,0.011556,50,0.009320,12,0.002237
1,光电工程学院,583,0.108667,476,0.088723,107,0.019944
2,兵器科学与技术学院,46,0.008574,0,NaN,46,0.008574
3,基础学院,272,0.050699,246,0.045853,26,0.004846
4,建筑工程学院,319,0.059459,255,0.047530,64,0.011929
5,文学院,323,0.060205,268,0.049953,55,0.010252
6,机电工程学院,675,0.125815,566,0.105499,109,0.020317
7,材料与化工学院,501,0.093383,404,0.075303,97,0.018080
8,电子信息工程学院,753,0.140354,637,0.118733,116,0.021622
9,经济管理学院,702,0.130848,586,0.109226,116,0.021622


In [ ]:
raise


In [ ]:
dict_sf = {}
list_sf = ['河北省',
'香港特别行政区',
'天津市',
'山西省',
'内蒙古自治区',
'四川省',
'海南省',
'江西省',
'北京市',
'黑龙江省',
'云南省',
'西藏自治区',
'浙江省',
'广西壮族自治区',
'辽宁省',
'澳门特别行政区',
'福建省',
'安徽省',
'青海省',
'贵州省',
'吉林省',
'湖南省',
'陕西省',
'台湾省',
'山东省',
'江苏省',
'重庆市',
'广东省',
'湖北省',
'甘肃省',
'新疆维吾尔自治区',
'宁夏回族自治区',
'上海市',
'河南省']
for i in list_sf:
    dict_sf[i] = i.strip('省市自治区壮族维吾尔回族特别行政')

In [ ]:
dict_sf

In [ ]:
def picture_china_map(data,picture_name,label_fontsize=13,color = get_lighter_color('#FFFFFF',color_1)[0]):
    data_zip = [*zip(data.index.tolist(),data)]#构建省份和对应统计人数的列表,地图数据传入的指定格式
    for i in range(len(data_zip)):
        data_zip[i]=[data_zip[i][0].strip('省市自治区壮族维吾尔回族特别行政'),data_zip[i][1]]
        data_zip[i][1]=i+1
    max_ = data_zip[len(data_zip)-1][1]
    a= (
    Map()
    .add("",data_zip,"china",is_map_symbol_show = False,is_roam = False,#数据传入,地图类型设置:中国地图"china"
         name_map=dict_sf,
         label_opts=opts.LabelOpts(formatter= '{b}',font_size=label_fontsize,font_family = '宋体'),#标签设置
         itemstyle_opts=opts.ItemStyleOpts(border_color = '#A6A6A6')) #图形的描边颜色
    .set_global_opts(
         visualmap_opts=opts.VisualMapOpts(is_show=False,max_ = max_,range_color = color))#颜色渐变设置
    )
    a = (
         Grid(init_opts = opts.InitOpts(height = '700px',width='1000px'))  #设置图形大小,高度:height,宽度:width
        .add(a, grid_opts=opts.GridOpts(pos_left='35%'))#位置调整
    )
    return make_snapshot(snapshot,a.render(), pic_path+f'{picture_name}.png',pixel_ratio = 5),a.render_notebook()

In [ ]:
raise

# 百分比四舍五入

In [ ]:
sum100([2,3,4])

# 获取排序顺序并对表进行排序

In [ ]:
group = {'default':'默认排序','xy':'按照给定的学院顺序排序','num_down':'按照人数降序','num_up':'按照人数升序'}
table = pd.read_excel('E:/桌面/test/表1.xlsx')
print(table.to_markdown())
result = get_order_way('排序规则：',group)
table = order_table(table,result,group)
table